In [1]:
import mysql.connector as connection
import pandas as pd
from kafka import KafkaProducer
import time
import numpy as np
from pyspark.sql.functions import *


In [2]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Convert to Spark DataFrame") \
    .getOrCreate()

24/05/14 11:19:44 WARN Utils: Your hostname, Bisheshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.163.225 instead (on interface en0)
24/05/14 11:19:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/14 11:19:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark_df = spark.read.csv('fc_account_master.csv', header=True, inferSchema=True)

In [4]:
spark_df.show(4)

+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
|     account_number|customer_code|product|product_category|acc_open_date|acc_closed_date|active_flag|
+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
|02XYZXYZ10015592101|    KL0255921|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015593701|    KL0255937|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015593801|    KL0255938|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015594801|    KL0255948|    SBA|           SBANU|   1999-03-22|           NULL|          0|
+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
only showing top 4 rows



In [13]:
a_df = spark_df.withColumn('encrypted_account_number', expr("base64(aes_encrypt(account_number, '1234567890abcdef', 'ECB', 'PKCS'))")).select('encrypted_account_number')
a_df.show(5)

+------------------------+
|encrypted_account_number|
+------------------------+
|    4EwF2NML3u4+LxvIf...|
|    vq8oHgffvNghw+1UE...|
|    vq8oHgffvNghw+1UE...|
|    1ZOETWlw5A6AowRKv...|
|    B+0mAHSQ/W1/ljK+X...|
+------------------------+
only showing top 5 rows



In [17]:

decrypted_df = a_df.withColumn('decrypted_account_number',
                               expr("trim(aes_decrypt(unbase64(encrypted_account_number), '1234567890abcdef', 'ECB', 'PKCS'))")).select('decrypted_account_number')

decrypted_df.show(5)


+------------------------+
|decrypted_account_number|
+------------------------+
|     02XYZXYZ10015592101|
|     02XYZXYZ10015593701|
|     02XYZXYZ10015593801|
|     02XYZXYZ10015594801|
|     02XYZXYZ10015597601|
+------------------------+
only showing top 5 rows



In [6]:
KAFKA_TOPIC_NAME_CONS = "FC_Account_Master3"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'

if __name__ == "__main__":
    print("Kafka Producer Application Started ... ")

    kafka_producer_obj = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS_CONS,
                                       value_serializer=lambda x: x.encode('utf-8'))
    
    spark_df = spark_df.withColumn('encrypted_account_number', expr("base64(aes_encrypt(account_number, '1234567890abcdef', 'ECB', 'PKCS'))"))

    
    data = spark_df.toPandas().to_dict(orient="records")
       

    message_list = []
    message = None
    for message in data:
        message_fields_value_list = []
        message_fields_value_list.append(message["encrypted_account_number"])
        message_fields_value_list.append(message["product"])
        message_fields_value_list.append(message["product_category"])

        message = ','.join(str(v) for v in message_fields_value_list)
        print("Message: ", message)
        kafka_producer_obj.send(KAFKA_TOPIC_NAME_CONS, message)
        time.sleep(1)


    print("Kafka Producer Application Completed. ")

Kafka Producer Application Started ... 
Message:  4EwF2NML3u4+LxvIfQcxJZKuI+6vCbpBmDem0b44lKw=,SBA,SBPPS
Message:  vq8oHgffvNghw+1UEhMiJ0/tqJJA9LOoDg3OFdXAekk=,SBA,SBPPS
Message:  vq8oHgffvNghw+1UEhMiJ+HvNQ4G+TEvaJdQAQ7nwH8=,SBA,SBPPS
Message:  1ZOETWlw5A6AowRKvr//EeHvNQ4G+TEvaJdQAQ7nwH8=,SBA,SBANU
Message:  B+0mAHSQ/W1/ljK+X3umOW3CGSaohAeqskXCjJMGGT4=,SBA,SBPPS
Message:  B+0mAHSQ/W1/ljK+X3umOU/tqJJA9LOoDg3OFdXAekk=,SBA,SBPPS
Message:  B+0mAHSQ/W1/ljK+X3umOeHvNQ4G+TEvaJdQAQ7nwH8=,SBA,SBPPS
Message:  6ldLZAQRlJOWQURd7dEgCDbTFtoJRtgM+tVN/0ZNPCY=,SBA,SBPPS
Message:  6ldLZAQRlJOWQURd7dEgCK0cnwziCo79Q1FQjy59bNI=,SBA,SBPPS
Message:  W+QlV2jO3cHvTWSEqx1xUsmmAQglUD17iX3Xyqwxdds=,SBA,SBPPS
Message:  zgBl65WWMRcsZsL17R9m3W7StD3UQsJuemp+s7SQhwE=,SBA,SBSTF
Message:  zgBl65WWMRcsZsL17R9m3TRXHALSYyTGRJ03Xb+3uqQ=,SBA,SBSTF
Message:  zgBl65WWMRcsZsL17R9m3Z3E+vGC7CW6kodT5Wo5BXk=,SBA,SBSTF
Message:  zgBl65WWMRcsZsL17R9m3QTjKTOLUy57DNccTDscPkA=,SBA,SBSTF
Message:  zgBl65WWMRcsZsL17R9m3Shn75LmMeLIx+QFu5j8

24/05/14 12:16:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 884620 ms exceeds timeout 120000 ms
24/05/14 12:16:10 WARN SparkContext: Killing executors is not supported by current scheduler.


Message:  oLS2VgxWDpD3AzLLdMxsiS6iKa2NRA9EOCd7tJ60moQ=,SBA,SBPPS


24/05/14 12:16:11 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

Message:  oLS2VgxWDpD3AzLLdMxsiSVpeEg4tUdytwyhkv4NxNQ=,SBA,SBPPS
Message:  oLS2VgxWDpD3AzLLdMxsidX7DZIt2HbWfgQsdUCkZqE=,SBA,SBGUR
Message:  oLS2VgxWDpD3AzLLdMxsiVfRYuUWu5pWFxXOEtTPAtI=,SBA,SBPPS
Message:  oLS2VgxWDpD3AzLLdMxsidbsXNhunKSrKDcaa/K9Mic=,SBA,SBGUR
Message:  oLS2VgxWDpD3AzLLdMxsiQu+UIiJygE/RPCsI7arT5Y=,SBA,SBPPS
Message:  oLS2VgxWDpD3AzLLdMxsiV88cVE76ONAPs1yXOEQA5k=,SBA,SBPPS
Message:  oLS2VgxWDpD3AzLLdMxsibr9cZWaygNS91YIEpSAaCk=,SBA,SBPPS
Message:  T3L9iawb7TKxbCo/yIbkmTBqa/kRJ392oBSYkt8xve0=,SBA,SBGUR
Message:  T3L9iawb7TKxbCo/yIbkmUmbFX25319BN/+XUWYblww=,SBA,SBGUR
Message:  T3L9iawb7TKxbCo/yIbkmSRaws2VH/dgP1fJ9+n70lM=,SBA,SBGUR


...

Message:  E97z6MmS7wv2ZB4L/z/jqnxpFNDnSTDmYY51QpHQaFY=,SBA,SBPPS
Message:  RaOXBO26LRxTdo9rDMiYtSOXBMncEgWxNTvQXt18IK0=,SBA,SBPPS
Message:  8opNVelRIPWhqlbguNSo43bM0ZUovAeFU79ouQTGmWM=,SBA,SBLBM
Message:  8opNVelRIPWhqlbguNSo44izQuYv2+NpesN25fQtjcE=,SBA,SBNCS
Message:  8opNVelRIPWhqlbguNSo4/WF0eR8otdnWoijM0t6yD0=,SBA,SBPPS
Message:  8opNVelRIPWhqlbguNSo489evO+JNqZtCrJCT5YGIgA=,SBA,SBPPS
Message:  8opNVelRIPWhqlbguNSo4wYO3egQ+Q596X5UBZY4q0Y=,SBA,SBPPS
Message:  8opNVelRIPWhqlbguNSo46besy0lzAY/YwNLCFXxrTs=,SBA,SBPPS
Message:  8opNVelRIPWhqlbguNSo447h3GKF/XKFoWZODwO+Z8E=,SBA,SBPPS


24/05/14 12:45:25 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

Message:  8opNVelRIPWhqlbguNSo4+HSuozsi5Dc0wrMU7Yr06w=,SBA,SBPPS
Message:  8opNVelRIPWhqlbguNSo43Q9W3UdlWWRGITTRIMkdV0=,SBA,SBPPS
Message:  8opNVelRIPWhqlbguNSo45PRc+F8e1JSqYEnJHNicFY=,SBA,SBEMP
Message:  YGM3taeWgQLQo//iVTgt0zg6nJyskkB5cyNYdtT0eKM=,SBA,SBEMP
Message:  YGM3taeWgQLQo//iVTgt0yWz0JWC4YROH5k64Sw8rK0=,SBA,SBGUR
Message:  E97z6MmS7wv2ZB4L/z/jqrCj/yPhPSn0tZck0yZ+0bs=,SBA,SBNCS
Message:  E97z6MmS7wv2ZB4L/z/jqqb9VfSXcVTtJltvPXKdqfU=,SBA,SBPPS
Message:  8opNVelRIPWhqlbguNSo48Ha/xmOL+axLs1E/2C0gC8=,SBA,SBPPS
Message:  YGM3taeWgQLQo//iVTgt08d8qNmI/ObfzbPMtxyYHCs=,SBA,SBLBM
Message:  E97z6MmS7wv2ZB4L/z/jqv4sFGifFYOr2y7G7XwZZmA=,SBA,SBPPS


24/05/14 12:45:35 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928)

Message:  E97z6MmS7wv2ZB4L/z/jqqbesy0lzAY/YwNLCFXxrTs=,SBA,SBPPS
Message:  E97z6MmS7wv2ZB4L/z/jqo7h3GKF/XKFoWZODwO+Z8E=,SBA,SBANU
Message:  8opNVelRIPWhqlbguNSo45ef7K8cI1iG+V9us3eQ16k=,SBA,SBNCS
Message:  gxQsmKPhIBsiRAsgljpWeITfiFHhkoXc1f1MHMkLIcU=,SBA,SBLBM
Message:  gxQsmKPhIBsiRAsgljpWeI5YZqaYvH14HhIiRs83rCk=,SBA,SBANU
Message:  E97z6MmS7wv2ZB4L/z/jqgpTzsrT33Hm3mWizWS6jTI=,SBA,SBNCS
Message:  8opNVelRIPWhqlbguNSo4y6ccH4Klqdj+VN23z+r45o=,SBA,SBLBM
Message:  8opNVelRIPWhqlbguNSo4x9xCvOKYoP+LBQUXyp16pI=,SBA,SBPPS
Message:  E97z6MmS7wv2ZB4L/z/jqnmKWgQh+pgl5NSiBgDYkCw=,SBA,SBPPS
Message:  gxQsmKPhIBsiRAsgljpWeAOF5hfv3knOoawqL5j7DVg=,SBA,SBLBM
Message:  E97z6MmS7wv2ZB4L/z/jqsUv3LjJZsqpFdkYECJ6LJI=,SBA,SBPPS
Message:  RaOXBO26LRxTdo9rDMiYtRpWt3RTyP190+6bm4ei3GA=,SBA,SBNCS
Message:  RaOXBO26LRxTdo9rDMiYteZuab1NGIsq5m1O86r8/cI=,SBA,SBLBM
Message:  IePg4bkHt0xLcjga9BDXpYuQu6EI85Wou0QBodnh/Lk=,SBA,SBNCS
Message:  IePg4bkHt0xLcjga9BDXpTOKrL3C+cThiFSk4MpZ1x0=,SBA,SBNCS
Message:  IePg4bkHt0xLcjg

In [ ]:
import pandas as pd
df3 = pd.read_csv('IRIS.csv')
df3.head()